"Uczenie maszynowe z użyciem Scikit-learn i Tensorflow" A. Geron, rozdział 2

## Pobranie danych

In [ ]:
import os
import tarfile
from six.moves import urllib

DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml/master/"
HOUSING_PATH = os.path.join("zestawy danych", "mieszkania")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

# pobranie danych
def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    if not os.path.isdir(housing_path):
        os.makedirs(housing_path)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()

In [ ]:
fetch_housing_data()

In [ ]:
import pandas as pd

def load_housing_data(housing_path=HOUSING_PATH):
    csv_path = os.path.join(housing_path, "housing.csv")
    return pd.read_csv(csv_path)

## Patrzymy

In [ ]:
housing = load_housing_data()
housing.head()

In [ ]:
housing.info()

In [ ]:
housing["ocean_proximity"].value_counts()

In [ ]:
housing.describe() # ignoruje nulle dla liczby sypialni

In [ ]:
# w total_bedrooms są nulle
# median income, median_age są normalizowane
# median_house_value zostało obcięte do max 50 000

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
housing.hist(bins=50, figsize=(20,15))
plt.show()

## Wyodrębnienie zbioru testowego

In [ ]:
import numpy as np

In [ ]:
# klasycznie
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

### Losowanie warstwowe

In [ ]:
housing["median_income"].hist()

In [ ]:
# Dzieli wartość przez 1,5 w celu ograniczenia liczby kategorii dochodów
housing["income_cat"] = np.ceil(housing["median_income"] / 1.5)
# Wartości przekraczające wartość 5 zostają przyporządkowane do klasy 5
housing["income_cat"].where(housing["income_cat"] < 5, 5.0, inplace=True)

In [ ]:
housing["income_cat"].value_counts()

In [ ]:
housing["income_cat"].hist()

In [ ]:
# podział na podstawie mediany dochodów (ważnej cechy (wiemy z wiedzy dziedzinowej))
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)
for train_index, test_index in split.split(housing, housing["income_cat"]):
    strat_train_set = housing.loc[train_index] # wybranie wierszy po indeksach
    strat_test_set = housing.loc[test_index]

In [ ]:
# sprawdzenie proporcji przynależności do klas ze wzgl. na medianę dochodów w danych, losowym i lepiej przygotowanym zbiorze testowym
def income_cat_proportions(data):
    return data["income_cat"].value_counts() / len(data)

train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)

compare_props = pd.DataFrame({
    "Łącznie": income_cat_proportions(housing),
    "L. warstwowe": income_cat_proportions(strat_test_set),
    "Losowe": income_cat_proportions(test_set),
}).sort_index()
compare_props["Błąd - losowe (%)"] = 100 * compare_props["Losowe"] / compare_props["Łącznie"] - 100
compare_props["Błąd - l. warstwowe (%)"] = 100 * compare_props["L. warstwowe"] / compare_props["Łącznie"] - 100

compare_props
# dla warstwowego super, dla losowego gorzej

In [ ]:
# usunięcie niepotrzebnej kolumny
for set_ in (strat_train_set, strat_test_set):
    set_.drop("income_cat", axis=1, inplace=True)

In [ ]:
# do testowych nie wolno nam już zaglądać

# Odkrywanie i wizualizacja

In [ ]:
housing = strat_train_set.copy()

In [ ]:
# Położenie punktów na mapie (Kalifornia)
housing.plot(kind="scatter", x="longitude", y="latitude", figsize=(10,7), 
             alpha=0.4,
             s=housing["population"]/100, label = "Populacja",
             c="median_house_value", cmap=plt.get_cmap("jet"), colorbar=True,
             sharex = False
            ) 
plt.legend()
# alpha – przezroczystość każdego punktu
# s – rozmiar koła
# c – kolor
# sharex=False usuwa błąd wyświetlania (nie były wyświetlane wartości osi x i legenda). Jest to tymczasowe rozwiązanie (patrz https://github.com/pandas-dev/pandas/issues/10611). 

Argument `sharex=False` usuwa błąd wyświetlania (nie były wyświetlane wartości osi x i legenda). Jest to tymczasowe rozwiązanie (patrz https://github.com/pandas-dev/pandas/issues/10611). Podziękowania dla Wilmera Arellano za wskazanie tego błędu.

In [ ]:
# import matplotlib.image as mpimg
# california_img=mpimg.imread(PROJECT_ROOT_DIR + '/rysunki/pierwszy_projekt/Kalifornia.png')
# ax = housing.plot(kind="scatter", x="Dł. geograficzna", y="Szer. geograficzna", figsize=(10,7),
#                        s=housing['Populacja']/100, label="Populacja",
#                        c="Mediana cen mieszkań", cmap=plt.get_cmap("jet"),
#                        colorbar=False, alpha=0.4,
#                       )
# plt.imshow(california_img, extent=[-124.55, -113.80, 32.45, 42.05], alpha=0.5)
# plt.ylabel("Szer. geograficzna", fontsize=14)
# plt.xlabel("Dł. geograficzna", fontsize=14)

# prices = housing["Mediana cen mieszkań"]
# tick_values = np.linspace(prices.min(), prices.max(), 11)
# cbar = plt.colorbar()
# cbar.ax.set_yticklabels(["$%dk"%(round(v/1000)) for v in tick_values], fontsize=14)
# cbar.set_label('Mediana cen mieszkań', fontsize=16)

# plt.legend(fontsize=16)
# save_fig("wykres_punktowy_cen_mieszkań_z_mapą_Kalifornii")
# plt.show()

In [ ]:
corr_matrix = housing.corr()

In [ ]:
# korelacja pearsona każdego atrybutu z medaną cen
corr_matrix["median_house_value"].sort_values(ascending=False)

In [ ]:
from pandas.plotting import scatter_matrix

attributes = ["median_house_value", "median_income", "total_rooms",
              "housing_median_age"]
scatter_matrix(housing[attributes], figsize=(12, 8))
plt.show()

# na przekątnej histogramy atrybutu (domyślnie)

In [ ]:
housing.plot(kind="scatter", x="median_income", y="median_house_value", alpha=0.1)
plt.axis([0, 16, 0, 550000])

# widać linię odcięcia na 50 000 oraz z linie w 45 000, 35 000, 28 000. Warto by było wyeliminować te anomalie

In [ ]:
# nowe cechy
housing["rooms_per_family"] = housing["total_rooms"]/housing["households"]
housing["bedrooms_per_rooms"] = housing["total_bedrooms"]/housing["total_rooms"]
housing["avg_family_size"]=housing["population"]/housing["households"]

In [ ]:
corr_matrix = housing.corr()
corr_matrix["median_house_value"].sort_values(ascending=False)

#bedrooms_per_rooms lepsze od total_bedrooms, rooms_per_family lepsze od total_rooms

In [ ]:
housing.plot(kind="scatter", x="rooms_per_family", y="median_house_value",
             alpha=0.2)
plt.axis([0, 5, 0, 520000])
plt.show()

In [ ]:
housing.describe()

# Przygotowanie danych pod algorytmy uczenia maszynowego

In [ ]:
housing = strat_train_set.drop("median_house_value", axis=1) # usuwa etykiety w zbiorze uczącym
housing_labels = strat_train_set["median_house_value"].copy()

### Brakujące cechy

In [ ]:
sample_incomplete_rows = housing[housing.isnull().any(axis=1)].head()
sample_incomplete_rows

In [ ]:
# sample_incomplete_rows.dropna(subset=["total_bedrooms"])    # I sp. - usunięcie wierszy z wart. pustymi
# sample_incomplete_rows.drop("total_bedrooms", axis=1)       # II sp. – usunięcie całej kolumny

median = housing["total_bedrooms"].median()
sample_incomplete_rows["total_bedrooms"].fillna(median, inplace=True) # III sp. – zastąpienie zerem, średnią lub medianą
sample_incomplete_rows

In [ ]:
# zautomatyzowanie uzupełniania
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='median')

Usuwamy atrybut tekstowy, ponieważ mediana może być obliczana wyłącznie wobec atrybutów numerycznych:

In [ ]:
housing_num = housing.drop('ocean_proximity', axis=1)
# housing_num = housing.select_dtypes(include=[np.number])

In [ ]:
# imputer.fit(housing_num) # wyliczenie median
# X = imputer.transform(housing_num) # podstawienie median pod nulle. Zwraca listę

X = imputer.fit_transform(housing_num)

In [ ]:
imputer.statistics_ # wyliczone mediany
# housing_num.median().values # tak byśmy to policzyli ręcznie

Przekształcamy zbiór uczący:

In [ ]:
housing_tr = pd.DataFrame(X, columns=housing_num.columns,
                          index = list(housing.index.values)) # przekształcenie listy do DF

In [ ]:
housing_tr.loc[sample_incomplete_rows.index.values]

### Dane kategorialne

In [ ]:
housing_cat = housing['ocean_proximity']

In [ ]:
housing_cat_encoded, housing_categories = housing_cat.factorize() # zamiana str na liczby stałoprzecinkowe
print(housing_cat_encoded[:10], housing_categories, sep='\n')

Nie jest to najlepsze rozwiązanie, bo algorytm może liczyć różnice między cechami, które nie mają praktycznego znaczenia.

Za pomocą klasy `OneHotEncoder` możemy przekształcić każdą wartość kategorialną do postaci wektora binarnego w którym tylko dla odpowiadającej wartości jest 1.

In [ ]:
from sklearn.preprocessing import OneHotEncoder

encoder = OneHotEncoder()
housing_cat_1hot = encoder.fit_transform(housing_cat_encoded.reshape(-1,1)) # zmiana wektora na macierz i zastosowanie oneHotEncoding
housing_cat_1hot

Klasa `OneHotEncoder` zwraca domyślnie macierz rzadką (żeby oszczędzać pamięć nie przechowując zer), ale w razie potrzeby możemy ją przekształcić do postaci gęstej: 

In [ ]:
#housing_cat_1hot.toarray()

In [ ]:
# Definicja klasy CategoricalEncoder, skopiowana z prośby PR #9151.
# Uruchom tę komórkę nie próbując rozumieć jej zawartości (na razie).

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.utils import check_array
from sklearn.preprocessing import LabelEncoder
from scipy import sparse

class CategoricalEncoder(BaseEstimator, TransformerMixin):
    """Koduje cechy kategorialne w postaci macierzy numerycznej.
    Danymi wejściowymi dostarczanymi do tego transformatora powinna być macierz
    zawierająca liczby stałoprzecinkowe lub ciągi znaków, symbolizujące
    wartości przechowywane przez cechy kategorialne (dyskretne).
    Możemy kodować cechy za pomocą schematu "gorącojedynkowego" (jeden-z-K)
    (``encoding='onehot'``, domyślne rozwiązanie) lub przekształcać je do postaci
    liczb porządkowych (``encoding='ordinal'``).
    Tego typu kodowanie jest wymagane podczas dostarczania danych kategorialnych do wielu
    etymatorów modułu Scikit-Learn, mianowicie w modelach liniowych i maszynach
    SVM wykorzystujących standardowe jądra. Więcej informacji znajdziesz w:
    :ref:`User Guide <preprocessing_categorical_features>`.
    Parametry
    ----------
    encoding : ciąg znaków, 'onehot', 'onehot-dense' lub 'ordinal'
        Rodzaj stosowanego kodowania (domyślna wartość to 'onehot'):
        - 'onehot': koduje cechy za pomocą schematu "gorącojedynkowego" (jeden-z-K,
           bywa również nazywany kodowaniem 'sztucznym'). Zostaje utworzona kolumna
           binarna dla każdej kategorii, a zwracana jest macierz rzadka.
        - 'onehot-dense': to samo, co wartość 'onehot', ale zwraca macierz gęstą zamiast rzadkiej.
        - 'ordinal': koduje cechy w postaci liczb porządkowych. Uzyskujemy w ten sposób 
          pojedynczą kolumną zawierającą liczby stałoprzecinkowe (od 0 do n_kategorii - 1) 
          dla każdej cechy.
    categories : 'auto' lub lista list/tablic wartości.
        Kategorie (niepowtarzalne wartości) na każdą cechę:
        - 'auto' : Automatycznie określa kategorie za pomocą danych uczących. 
        - lista : ``categories[i]`` przechowuje kategorie oczekiwane w i-tej kolumnie.
          Przekazane kategorie zostają posortowanie przed zakodowaniem danych
          (użyte kategorie można przejrzeć w atrybucie ``categories_``).
    dtype : typ liczby, domyślnie np.float64
        Wymagany typ wartości wyjściowej.
    handle_unknown : 'error' (domyślnie) lub 'ignore'
        Za jego pomocą określamy, czy w przypadku obecności nieznanej cechy w czasie
        wykonywania transformacji ma być wyświetlany komunikat o błędzie (wartość
        domyślna) lub czy ma zostać zignorowana. Po wybraniu wartości 'ignore' 
        i natrafieniu na nieznaną kategorię w trakcie przekształceń, wygenerowane
        kolumny "gorącojedynkowe" dla tej cechy będą wypełnione zerami. 
        Ignorowanie nieznanych kategorii nie jest obsługiwane w parametrze
        ``encoding='ordinal'``.
    Atrybuty
    ----------
    categories_ : lista tablic
        Kategorie każdej cechy określone podczas uczenia. W przypadku ręcznego 
        wyznaczania kategorii znajdziemy tu listę posortowanych kategorii
        (w kolejności odpowiadającej wynikowi operacji 'transform').
    Przykłady
    --------
    Mając zbiór danych składający się z trzech cech i dwóch próbek pozwalamy koderowi
    znaleźć maksymalną wartość każdej cechy i przekształcić dane do postaci
    binarnego kodowania "gorącojedynkowego".
    >>> from sklearn.preprocessing import CategoricalEncoder
    >>> enc = CategoricalEncoder(handle_unknown='ignore')
    >>> enc.fit([[0, 0, 3], [1, 1, 0], [0, 2, 1], [1, 0, 2]])
    ... # doctest: +ELLIPSIS
    CategoricalEncoder(categories='auto', dtype=<... 'numpy.float64'>,
              encoding='onehot', handle_unknown='ignore')
    >>> enc.transform([[0, 1, 1], [1, 0, 4]]).toarray()
    array([[ 1.,  0.,  0.,  1.,  0.,  0.,  1.,  0.,  0.],
           [ 0.,  1.,  1.,  0.,  0.,  0.,  0.,  0.,  0.]])
    Powiązane materiały
    --------
    sklearn.preprocessing.OneHotEncoder : przeprowadzana kodowanie "gorącojedynkowe"
      stałoprzecinkowych cech porządkowych. Klasa ``OneHotEncoder zakłada``, że cechy wejściowe
      przechowują wartości w zakresie ``[0, max(cecha)]`` zamiast korzystać z
      niepowtarzalnych wartości.
    sklearn.feature_extraction.DictVectorizer : przeprowadzana kodowanie "gorącojedynkowe"
      elementów słowanika (a także cech przechowujących ciągi znaków).
    sklearn.feature_extraction.FeatureHasher : przeprowadzana przybliżone kodowanie "gorącojedynkowe"
      elementów słownika lub ciągów znaków.
    """

    def __init__(self, encoding='onehot', categories='auto', dtype=np.float64,
                 handle_unknown='error'):
        self.encoding = encoding
        self.categories = categories
        self.dtype = dtype
        self.handle_unknown = handle_unknown

    def fit(self, X, y=None):
        """Dopasowuje klasę CategoricalEncoder do danych wejściowych X.
        Parametry
        ----------
        X : tablicopodobny, postać [n_próbek, n_cech]
            Dane służące do określania kategorii każdej cechy.
        Zwraca
        -------
        self
        """

        if self.encoding not in ['onehot', 'onehot-dense', 'ordinal']:
            template = ("Należy wybrać jedno z następujących kodowań: 'onehot', 'onehot-dense' "
                        "lub 'ordinal', wybrano %s")
            raise ValueError(template % self.handle_unknown)

        if self.handle_unknown not in ['error', 'ignore']:
            template = ("Należy wybrać jedną z następujących wartości parametru handle_unknown: 'error' lub "
                        "'ignore', wybrano %s")
            raise ValueError(template % self.handle_unknown)

        if self.encoding == 'ordinal' and self.handle_unknown == 'ignore':
            raise ValueError("Wartość handle_unknown='ignore' nie jest obsługiwana przez parametr"
                             " encoding='ordinal'")

        X = check_array(X, dtype=object, accept_sparse='csc', copy=True)
        n_samples, n_features = X.shape

        self._label_encoders_ = [LabelEncoder() for _ in range(n_features)]

        for i in range(n_features):
            le = self._label_encoders_[i]
            Xi = X[:, i]
            if self.categories == 'auto':
                le.fit(Xi)
            else:
                valid_mask = np.in1d(Xi, self.categories[i])
                if not np.all(valid_mask):
                    if self.handle_unknown == 'error':
                        diff = np.unique(Xi[~valid_mask])
                        msg = ("Znaleziono nieznane kategorie {0} w kolumnie {1}"
                               " podczas dopasowywania".format(diff, i))
                        raise ValueError(msg)
                le.classes_ = np.array(np.sort(self.categories[i]))

        self.categories_ = [le.classes_ for le in self._label_encoders_]

        return self

    def transform(self, X):
        """Przekształca X za pomocą kodowania "gorącojedynkowego".
        Parametry
        ----------
        X : tablicopodobny, postać [n_próbek, n_cech]
            Kodowane dane.
        Zwraca
        -------
        X_out : macierz rzadka lub dwuwymiarowa tablica
            Przekształcone dane wejściowe.
        """
        X = check_array(X, accept_sparse='csc', dtype=object, copy=True)
        n_samples, n_features = X.shape
        X_int = np.zeros_like(X, dtype=int)
        X_mask = np.ones_like(X, dtype=bool)

        for i in range(n_features):
            valid_mask = np.in1d(X[:, i], self.categories_[i])

            if not np.all(valid_mask):
                if self.handle_unknown == 'error':
                    diff = np.unique(X[~valid_mask, i])
                    msg = ("Znaleziono nieznane kategorie {0} w kolumnie {1}"
                           " podczas przekształcania".format(diff, i))
                    raise ValueError(msg)
                else:
                    # Wyznaczamy akceptowalną wartość rzędom sprawiającym problem i
                    # kontynuujemy. Rzędy te zostają oznaczone jako `X_mask` i zostaną
                    # później usunięte.
                    X_mask[:, i] = valid_mask
                    X[:, i][~valid_mask] = self.categories_[i][0]
            X_int[:, i] = self._label_encoders_[i].transform(X[:, i])

        if self.encoding == 'ordinal':
            return X_int.astype(self.dtype, copy=False)

        mask = X_mask.ravel()
        n_values = [cats.shape[0] for cats in self.categories_]
        n_values = np.array([0] + n_values)
        indices = np.cumsum(n_values)

        column_indices = (X_int + indices[:-1]).ravel()[mask]
        row_indices = np.repeat(np.arange(n_samples, dtype=np.int32),
                                n_features)[mask]
        data = np.ones(n_samples * n_features)[mask]

        out = sparse.csc_matrix((data, (row_indices, column_indices)),
                                shape=(n_samples, indices[-1]),
                                dtype=self.dtype).tocsr()
        if self.encoding == 'onehot-dense':
            return out.toarray()
        else:
            return out

Klasa `CategoricalEncoder` oczekuje dwuwymiarowej tablicy zawierającej co najmniej jedną kategorialną cechę wejściową. Musimy przekształcić dane `housing_cat` do postaci duwywmiarowej tablicy:

In [ ]:
# from sklearn.preprocessing import CategoricalEncoder # w przyszłych wersjach modułu Scikit-Learn

cat_encoder = CategoricalEncoder()
housing_cat_reshaped = housing_cat.values.reshape(-1, 1)
housing_cat_1hot = cat_encoder.fit_transform(housing_cat_reshaped)
housing_cat_1hot

Domyślne jest stosowane kodowanie "gorącojedynkowe", które zwraca macierz rzadką. Możemy ją przekształcić do formy gęstej przy użyciu metody `toarray()`. Ewentualnie możemy wyznaczyć parametr kodowania `"onehot-dense"`, aby od razu uzyskać macierz gęstą:

In [ ]:
cat_encoder = CategoricalEncoder(encoding="onehot-dense")
housing_cat_1hot = cat_encoder.fit_transform(housing_cat_reshaped)
housing_cat_1hot

In [ ]:
cat_encoder.categories_

### Niestandardowy transformator wstawiający dodatkowe atrybuty

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# Indeks kolumny
rooms_ix, bedrooms_ix, population_ix, household_ix = 3, 4, 5, 6

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    # transformerMixin daje metodę fit_transform
    # base estimator – get params i set_params (o ile nie używamu args i kwargs)
    def __init__(self, add_bedrooms_per_room = True): # żadnych zmiennych *args ani **kargs
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nie robi nic innego
    def transform(self, X, y=None):
        rooms_per_family = X[:, rooms_ix] / X[:, household_ix]
        population_per_family = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_family, population_per_family,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_family, population_per_family]

attr_adder = CombinedAttributesAdder(add_bedrooms_per_room=False)
housing_extra_attribs = attr_adder.transform(housing.values)

In [ ]:
housing_extra_attribs = pd.DataFrame(housing_extra_attribs, columns=list(housing.columns)+["rooms_per_family", "population_per_family"])
housing_extra_attribs.head()

Stwórzmy teraz potok służący do wstępnej obróbki atrybutów numerycznych:

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

num_pipeline = Pipeline([
        ('imputer', SimpleImputer(strategy="median")),  # wypełnienie wartości pustych
        ('attribs_adder', CombinedAttributesAdder()), # dodanie nowych atrybutów
        ('std_scaler', StandardScaler()), # standaryzacja – odjęcie średniej i podzielenie przez wariancję
        # nie ma normalizacji (skalowania min-max) do przedziału 0-1
    ])

housing_num_tr = num_pipeline.fit_transform(housing_num)

A także transformator służący jedynie do wybierania podzbioru kolumn obiektu DataFrame:

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

# Tworzy klasę wybierającą numeryczne i kategorialne kolumny,
# gdyż moduł Scikit-Learn nie zawiera jeszcze obsługi obiektów DataFrame
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribute_names):
        self.attribute_names = attribute_names
        
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X[self.attribute_names].values

Połączmy teraz te składowe w jeden duży potok przetwarzający wstępnie zarówno cechy numeryczne, jak i kategorialne:

In [ ]:
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
        ('selector', DataFrameSelector(num_attribs)), # wybranie atrybutów numerycznych
        ('imputer', SimpleImputer(strategy="median")),  # wypełnienie wartości pustych
        ('attribs_adder', CombinedAttributesAdder()), # dodanie nowych atrybutów
        ('std_scaler', StandardScaler()), # standaryzacja – odjęcie średniej i podzielenie przez wariancję
        # nie ma normalizacji (skalowania min-max) do przedziału 0-1
    ])

cat_pipeline = Pipeline([
        ('selector', DataFrameSelector(cat_attribs)), # wybranie atrybutów kategorialnych
        ('cat_encoder', CategoricalEncoder(encoding="onehot-dense")), # one hot encoding z wynikiem w postaci gęstej macierzy
    ])

In [ ]:
from sklearn.pipeline import FeatureUnion

# wykonanie równolegle 2 potoków

full_pipeline = FeatureUnion(transformer_list=[
        ("num_pipeline", num_pipeline),
        ("cat_pipeline", cat_pipeline),
    ])

In [ ]:
housing_prepared = full_pipeline.fit_transform(housing)
# użycie samej metody fit() spowodowałoby wywołanie fit_transform na wszystkich elementach potoku oprócz ostatniego
# i przekazanie wyniku dalej, i użycie fit() na ostatnim estymatorze.

housing_prepared

In [ ]:
housing_prepared.shape

# Dobór i uczenie modelu

In [ ]:
# REGRESJA LINIOWA

from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error

lin_reg = LinearRegression()
lin_reg.fit(housing_prepared, housing_labels)
housing_predictions = lin_reg.predict(housing_prepared)

lin_mse = mean_squared_error(housing_labels, housing_predictions)
lin_rmse = np.sqrt(lin_mse)

lin_mae = mean_absolute_error(housing_labels, housing_predictions)

print("rmse:", lin_rmse, "\nmae:", lin_mae)

# To duży błąd patrząc na zakres wartości cen domów (Q1=120 000  Q3=265 000). Model jest niedotrenowany – za prosty

In [ ]:
# DRZEWO DECYZYJNE

from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor(random_state=42)
tree_reg.fit(housing_prepared, housing_labels)

housing_predictions = tree_reg.predict(housing_prepared)

tree_mse = mean_squared_error(housing_labels, housing_predictions)
tree_rmse = np.sqrt(tree_mse)

tree_mae = mean_absolute_error(housing_labels, housing_predictions)

print("rmse:", tree_rmse, "\nmae:", tree_mae)

# Podejrzane. Przetrenowanie?

# Regulowanie modelu

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(tree_reg, housing_prepared, housing_labels,
                         scoring="neg_mean_squared_error", cv=10)
tree_rmse_scores = np.sqrt(-scores) # scores to przeciwieństwo mse

In [ ]:
def display_scores(scores):
    print("Wyniki:", scores) # 10 wyników dla 10 testów
    print("Średnia:", scores.mean())
    print("Odchylenie standardowe:", scores.std())

display_scores(tree_rmse_scores)

In [ ]:
lin_scores = cross_val_score(lin_reg, housing_prepared, housing_labels,
                             scoring="neg_mean_squared_error", cv=10)
lin_rmse_scores = np.sqrt(-lin_scores)
display_scores(lin_rmse_scores)

# błąd na zbiorze testowym dla drzew decyzyjnych gorszy niż dla regresji liniowej. Przetrenowaliśmy model :(

In [ ]:
# LAS LOSOWY – wiele uśrednionych wyników drzew
# długo się trenuje

from sklearn.ensemble import RandomForestRegressor

forest_reg = RandomForestRegressor(random_state=42)
forest_reg.fit(housing_prepared, housing_labels)

In [ ]:
housing_predictions = forest_reg.predict(housing_prepared)
forest_mse = mean_squared_error(housing_labels, housing_predictions)
forest_rmse = np.sqrt(forest_mse)
forest_rmse

In [ ]:
forest_scores = cross_val_score(forest_reg, housing_prepared, housing_labels,
                                scoring="neg_mean_squared_error", cv=10)
forest_rmse_scores = np.sqrt(-forest_scores)
display_scores(forest_rmse_scores)

# błąd na zbiorze walidacyjnym wciąż sporo wyższy niż na uczącym, trochę przetrenowany jeszcze jest – trzeba dobrać parametry

In [ ]:
# Support vectore machines
from sklearn.svm import SVR

svm_reg = SVR(kernel="linear")
svm_reg.fit(housing_prepared, housing_labels)
housing_predictions = svm_reg.predict(housing_prepared)
svm_mse = mean_squared_error(housing_labels, housing_predictions)
svm_rmse = np.sqrt(svm_mse)
svm_rmse

# wygląda obiecująco

Tu szukalibyśmy jeszcze obiecujących modeli, do dostrojenia...

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
    # wypróbowuje 12 (3×4) kombinacji hiperparametrów
    {'n_estimators': [3, 10, 30], 'max_features': [2, 4, 6, 8]},
    # następnie wypróbowuje 6 (2×3) kombinacji z wyłączonym parametrem bootstrap (False)
    {'bootstrap': [False], 'n_estimators': [3, 10], 'max_features': [2, 3, 4]},
  ]

forest_reg = RandomForestRegressor(random_state=42)
# przeprowadza proces uczenia wobec pięciu podzbioru, czyli łącznie (12+6)*5=90 przebiegów 
grid_search = GridSearchCV(forest_reg, param_grid, cv=5,
                           scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(housing_prepared, housing_labels)

Najlepsza znaleziona kombinacja hiperparametrów:

In [ ]:
grid_search.best_params_

In [ ]:
grid_search.best_estimator_

Przyjrzyjmy się wynikowi każdej kombinacji hiperparametrów przetestowanej w trakcie przeszukiwania siatki:

In [ ]:
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
pd.DataFrame(grid_search.cv_results_)

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

param_distribs = {
        'n_estimators': randint(low=1, high=200),
        'max_features': randint(low=1, high=8),
    }

forest_reg = RandomForestRegressor(random_state=42)
rnd_search = RandomizedSearchCV(forest_reg, param_distributions=param_distribs,
                                n_iter=10, cv=5, scoring='neg_mean_squared_error', random_state=42)
rnd_search.fit(housing_prepared, housing_labels)

In [ ]:
cvres = rnd_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"], cvres["params"]):
    print(np.sqrt(-mean_score), params)

In [ ]:
feature_importances = grid_search.best_estimator_.feature_importances_
feature_importances

In [ ]:
extra_attribs = ["Pokoje_na_rodzinę", "Populacja_na_rodzinę", "Sypialnie_na_pokoje"]
cat_encoder = cat_pipeline.named_steps["cat_encoder"]
cat_one_hot_attribs = list(cat_encoder.categories_[0])
attributes = num_attribs + extra_attribs + cat_one_hot_attribs
sorted(zip(feature_importances, attributes), reverse=True)

In [ ]:
final_model = grid_search.best_estimator_

X_test = strat_test_set.drop("Mediana cen mieszkań", axis=1)
y_test = strat_test_set["Mediana cen mieszkań"].copy()

X_test_prepared = full_pipeline.transform(X_test)
final_predictions = final_model.predict(X_test_prepared)

final_mse = mean_squared_error(y_test, final_predictions)
final_rmse = np.sqrt(final_mse)

In [ ]:
final_rmse

# Materiały dodatkowe

## Pełen potok zawierający etapy przygotowania i prognozowania

In [ ]:
full_pipeline_with_predictor = Pipeline([
        ("preparation", full_pipeline),
        ("linear", LinearRegression())
    ])

full_pipeline_with_predictor.fit(housing, housing_labels)
full_pipeline_with_predictor.predict(some_data)

## Zapisanie stanu modelu za pomocą narzędzia joblib

In [ ]:
my_model = full_pipeline_with_predictor

In [ ]:
from sklearn.externals import joblib
joblib.dump(my_model, "mój_model.pkl") # DIFF
#...
my_model_loaded = joblib.load("mój_model.pkl") # DIFF

## Przykładowe rozkłady w module SciPy za pomocą klasy `RandomizedSearchCV`

In [ ]:
from scipy.stats import geom, expon
geom_distrib=geom(0.5).rvs(10000, random_state=42)
expon_distrib=expon(scale=1).rvs(10000, random_state=42)
plt.hist(geom_distrib, bins=50)
plt.show()
plt.hist(expon_distrib, bins=50)
plt.show()

# Rozwiązania ćwiczeń

## 1.

Pytanie: Wypróbuj regresor maszyny wektorów nośnych (`sklearn.svm.SVR`) przy użyciu różnych hiperparametrów, takich jak `kernel="linear"` (oraz różnych wartości hiperparametru `C`) lub `kernel="rbf"` (oraz różnych wartości hiperparametrów `C` i `gamma`). Na razie nie przejmuj się tym, że nie wiesz, do czego te hiperparametry służą. Jak się spisuje najlepszy predyktor maszyny wektorów nośnych?

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = [
        {'kernel': ['linear'], 'C': [10., 30., 100., 300., 1000., 3000., 10000., 30000.0]},
        {'kernel': ['rbf'], 'C': [1.0, 3.0, 10., 30., 100., 300., 1000.0],
         'gamma': [0.01, 0.03, 0.1, 0.3, 1.0, 3.0]},
    ]

svm_reg = SVR()
grid_search = GridSearchCV(svm_reg, param_grid, cv=5, scoring='neg_mean_squared_error', verbose=2, n_jobs=4)
grid_search.fit(housing_prepared, housing_labels)

Najlepszy model uzyskuje następujący wynik (wyliczony za pomocą pięciokrotnego sprawdzianu krzyżowego): 

In [ ]:
negative_mse = grid_search.best_score_
rmse = np.sqrt(-negative_mse)
rmse

Jest to znacznie gorszy rezultat od uzyskanego za pomocą klasy `RandomForestRegressor`. Sprawdźmy, jakie najlepsze hiperparametry zostały znalezione:

In [ ]:
grid_search.best_params_

Jądro liniowe wydaje się być lepsze od jądra RBF. Zwróć uwagę, że wartość parametru `C` stanowi maksymalną przetestowaną wartość. W takiej sytuacji zdecydowanie należy uruchomić ponownie przeszukiwanie siatki z większymi wartościami tego hiperparametru (usuwając jednocześnie mniejsze wartości), ponieważ prawdopodobnie któraś z nich będzie jeszcze lepiej dostosowana do modelu.

## 2.

Pytanie: Spróbuj zastąpić klasę `GridSearchCV` klasą `RandomizedSearchCV`.

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import expon, reciprocal

# na stronie https://docs.scipy.org/doc/scipy-0.19.0/reference/stats.html
# znajdziesz dokumentację funkcji `expon()` i `reciprocal()`, a także innych funkcji rozkładu prawdopodobieństwa.

# Uwaga: parametr gamma jest ignorowany, gdy korzystamy z jądra "linear"
param_distribs = {
        'kernel': ['linear', 'rbf'],
        'C': reciprocal(20, 200000),
        'gamma': expon(scale=1.0),
    }

svm_reg = SVR()
rnd_search = RandomizedSearchCV(svm_reg, param_distributions=param_distribs,
                                n_iter=50, cv=5, scoring='neg_mean_squared_error',
                                verbose=2, n_jobs=4, random_state=42)
rnd_search.fit(housing_prepared, housing_labels)

Najlepszy model uzyskuje następujący wynik (obliczony za pomocą pięciokrotnego sprawdzianu krzyżowego):

In [ ]:
negative_mse = rnd_search.best_score_
rmse = np.sqrt(-negative_mse)
rmse

Teraz uzyskaliśmy rezultat o wiele bardziej zbliżony do wydajności klasy `RandomForestRegressor` (ciągle mu jednak trochę brakuje). Przyjrzyjmy się znalezionym hiperparametrom:

In [ ]:
rnd_search.best_params_

Tym razem udało się znaleźć dobry zestaw hiperparametrów dla jądra RBF. Przeszukiwanie losowe zazwyczaj jest w stanie wyszukiwać w tym samym czasie lepsze hiperparametry od przeszukiwania siatki.

Sprawdźmy wykorzystany przez rozkład wykładniczy, wykorzystujący parametr `scale=1.0`. Zwróć uwagę, że niektóre próbki są znacznie większe lub mniejsze od wartości 1.0, ale gdy sprawdzimy logarytm tego rozkładu, zauważymy, że większość wartości mieści się głównie w przedziale od exp(-2) do exp(+2), czyli od mniej więcej 0.1 do 7.4.

In [ ]:
expon_distrib = expon(scale=1.)
samples = expon_distrib.rvs(10000, random_state=42)
plt.figure(figsize=(10, 4))
plt.subplot(121)
plt.title("Rozkład wykładniczy (scale=1.0)")
plt.hist(samples, bins=50)
plt.subplot(122)
plt.title("Logarytm tego rozkładu")
plt.hist(np.log(samples), bins=50)
plt.show()

Rozkład użyty dla hiperparametru `C` wygląda całkiem inaczej: skala próbek została wyznaczona z rozkładu jednorodnego o określonym zakresie, co stanowi powód, że wykres widoczny po prawej (reprezentujący logarytm rozkładu) jest mniej więcej jednolity. Rozkład ten przydaje się, gdy nie mamy pojęcia, jaka jest docelowa skala:

In [ ]:
reciprocal_distrib = reciprocal(20, 200000)
samples = reciprocal_distrib.rvs(10000, random_state=42)
plt.figure(figsize=(10, 4))
plt.subplot(121)
plt.title("Rozkład odwrotny (scale=1.0)")
plt.hist(samples, bins=50)
plt.subplot(122)
plt.title("Logarytm tego rozkładu")
plt.hist(np.log(samples), bins=50)
plt.show()

Rozkład odwrotny przydaje się wtedy, gdy nie wiemy, w jakiej skali powinien znajdować się dany hiperparametr (rzeczywiście, na prawym wykresie widzimy, że wszystkie skale mają mniej więcej takie samo prawdopodobobieństwo w określonym zakresie), natomiast rozkład wykładniczy nadaje się w sytuacjach, gdy w przybliżeniu znamy skalę hiperparametru.

## 3.

Pytanie: 3.	Spróbuj dodać w potoku przygotowawczym funkcję przekształcającą w taki sposób, aby były dobierane wyłącznie najistotniejsze atrybuty.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin

def indices_of_top_k(arr, k):
    return np.sort(np.argpartition(np.array(arr), -k)[-k:])

class TopFeatureSelector(BaseEstimator, TransformerMixin):
    def __init__(self, feature_importances, k):
        self.feature_importances = feature_importances
        self.k = k
    def fit(self, X, y=None):
        self.feature_indices_ = indices_of_top_k(self.feature_importances, self.k)
        return self
    def transform(self, X):
        return X[:, self.feature_indices_]

Uwaga: Omawiany selektor cech zakłada, że już w jakiś sposób obliczyliśmy istotności cech (np. za pomocą klasy `RandomForestRegressor`). Być może kusi Cię, aby obliczyć je bezpośrednio w metodzie fit() klasy `TopFeatureSelector`, ale rozwiązanie to prawdopodobnie spowolniłoby proces przeszukiwania siatki/losowego, ponieważ istotności cech muszą zostać wyliczone dla każdej kombinacji hiperparametrów (chyba że zaimplementujesz jakąś formę pamięci podręcznej).

Zdefiniujmy liczbę najlepszych cech, które chcemy przechować:

In [ ]:
k = 5

Przyjrzyjmy się teraz indeksom k najlepszych cech:

In [ ]:
top_k_feature_indices = indices_of_top_k(feature_importances, k)
top_k_feature_indices

In [ ]:
np.array(attributes)[top_k_feature_indices]

Upewnijmy się, że mamy rzeczywiście do czynienia z k najlepszymi cechami:

In [ ]:
sorted(zip(feature_importances, attributes), reverse=True)[:k]

Wygląda nieźle... Stwórzmy teraz nowy potok, w którym połączymy wcześniej zdefiniowany potok przygotowawczy z doborem k najlepszych cech:

In [ ]:
preparation_and_feature_selection_pipeline = Pipeline([
    ('preparation', full_pipeline),
    ('feature_selection', TopFeatureSelector(feature_importances, k))
])

In [ ]:
housing_prepared_top_k_features = preparation_and_feature_selection_pipeline.fit_transform(housing)

Przyjrzyjmy się cechom trzech pierwszych próbek:

In [ ]:
housing_prepared_top_k_features[0:3]

Upewnijmy się, że mamy do czynienia z k najlepszymi cechami: 

In [ ]:
housing_prepared[0:3, top_k_feature_indices]

Wszystko śmiga!  :)

## 4.

Pytanie: 4.	Spróbuj stworzyć pojedynczy potok przeprowadzający pełne przygotowywanie danych i generujący ostateczne prognozy.

In [ ]:
prepare_select_and_predict_pipeline = Pipeline([
    ('preparation', full_pipeline),
    ('feature_selection', TopFeatureSelector(feature_importances, k)),
    ('svm_reg', SVR(**rnd_search.best_params_))
])

In [ ]:
prepare_select_and_predict_pipeline.fit(housing, housing_labels)

Wypróbujmy pełen potok na kilku próbkach:

In [ ]:
some_data = housing.iloc[:4]
some_labels = housing_labels.iloc[:4]

print("Prognozy:\t", prepare_select_and_predict_pipeline.predict(some_data))
print("Etykiety:\t\t", list(some_labels))

Wygląda na to, że pełen potok działa prawidłowo. Prognozy, oczywiście, nie są jakieś wybitne: byłyby lepsze, gdybyśmy użyli najlepszego znalezionego wcześniej regresora `RandomForestRegressor` zamiast `SVR`.

## 5.

Pytanie: 1.	Sprawdź automatycznie niektóre funkcje przygotowawcze za pomocą klasy `GridSearchCV`.

In [ ]:
param_grid = [
        {'preparation__num_pipeline__imputer__strategy': ['mean', 'median', 'most_frequent'],
         'feature_selection__k': list(range(1, len(feature_importances) + 1))}
]

grid_search_prep = GridSearchCV(prepare_select_and_predict_pipeline, param_grid, cv=5,
                                scoring='neg_mean_squared_error', verbose=2, n_jobs=4)
grid_search_prep.fit(housing, housing_labels)

In [ ]:
grid_search_prep.best_params_

Najlepszą strategią klasy imputer jest most_frequent i najwidoczniej niemal wszystkie cechy okazują się przydatne (15 z 16). Ostatnia cecha (ISLAND) prawdopodobnie wprowadza jedynie dodatkowy szum.

Gratulacje! Wiesz już całkiem dużo na temat uczenia maszynowego. :) You already know quite a lot about Machine Learning. :)